<a href="https://www.kaggle.com/code/realshaktigupta/decision-tree-on-moons-dataset?scriptVersionId=131486212" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Generating The Dataset**

First,we generate the dataset with a sample size of 10000 and some noise(0.4 to be precise).

In [2]:
from sklearn.datasets import make_moons
moons_data=make_moons(n_samples=10000,noise=0.4)

In [3]:
moons_data[0][0]

array([0.1354489 , 1.25093156])

The dataset is a tuple of 2D-lists.The first element in each list is the feature value and the 2nd one is the target value. Let's create feature set X and target set y and then split it into test and train sets.

In [4]:
X=moons_data[0]
y=moons_data[1]

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

Now we use GridsearchCV to determine the best hyperparameters for our decision tree model.

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
decision_tree_clf= DecisionTreeClassifier()
param_grid={'criterion':["gini","entropy","log_loss"],'splitter':["best","random"],
            'max_depth':[None,2,3,5,7,10,13,14],'min_samples_split':[2,3,4],
            'max_leaf_nodes':[5, 7, 10, 13, 15,16,17,18]}
grid_search = GridSearchCV(decision_tree_clf, param_grid,verbose=1, n_jobs=-1, cv=3)

In [7]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 1152 candidates, totalling 3456 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 2, 3, 5, 7, 10, 13, 14],
                         'max_leaf_nodes': [5, 7, 10, 13, 15, 16, 17, 18],
                         'min_samples_split': [2, 3, 4],
                         'splitter': ['best', 'random']},
             verbose=1)

In [8]:
grid_search.best_score_

0.8604999018783568

In [9]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': None,
 'max_leaf_nodes': 15,
 'min_samples_split': 2,
 'splitter': 'best'}

In [10]:
grid_search.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=15)

Now,we train on the entire training set using these best hyperparameters.

In [11]:
best_decision_tree_clf=DecisionTreeClassifier(criterion='entropy',max_depth=None,max_leaf_nodes=15,
min_samples_split=2,splitter='best')
best_decision_tree_clf.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=15)

In [12]:
y_pred=best_decision_tree_clf.predict(X_test)

In [13]:
from sklearn.metrics import mean_squared_error
accuracy=1-mean_squared_error(y_pred,y_test)
accuracy

0.8445

So,our model has an accuracy of 85.25%. This figure can vary upto +-2% depending on the random dataset generated.